In [1]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

In [2]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [3]:
# Fetch the sdRDM schema
lib = DataModel.from_markdown("./model.md")
lib.Root.meta_tree()

Root
├── id
├── value
├── nested
│   └── 0
│       ├── id
│       └── another_value
└── some_values


ClassNode(/Root, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [5]:
# Create tables for the sdRDM schema
create_tables(db_connector=db, model=lib.Root)

In [6]:
db.connection.tables

Tables
------
- Root
- nested
- some_values

In [7]:
# We are now seeking a way to insert an instance of 
# the sdRDM schema into the database.
print(db.connection.table("Root"))
print(db.connection.table("nested"))
print(db.connection.table("some_values"))

┏━━━━━━━━━┳━━━━━━━━━┓
┃ value   ┃ Root_id ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│ float64 │ !string │
└─────────┴─────────┘



┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┓
┃ another_value ┃ nested_id ┃ Root_id ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━┩
│ string        │ !string   │ string  │
└───────────────┴───────────┴─────────┘



┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ some_values ┃ Root_id ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ !string     │ string  │
└─────────────┴─────────┘



In [8]:
dataset1 = lib.Root(value=100.0, some_values=["Hello", "World"])
dataset1.add_to_nested(another_value="something")
dataset1.add_to_nested(another_value="something else")

print(dataset1)

Root
├── id = root0
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested0
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = nested1
│           └── another_value = something else
└── some_values = [Hello, World, ...]



In [9]:
dataset2 = lib.Root(value=20.0, some_values=["Whats", "up"])
dataset2.add_to_nested(another_value="hi there")
dataset2.add_to_nested(another_value="this is dataset 2")

print(dataset2)

Root
├── id = root1
├── value = 20.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested2
│   │       └── another_value = hi there
│   └── 1
│       └── Nested
│           ├── id = nested3
│           └── another_value = this is dataset 2
└── some_values = [Whats, up, ...]



In [10]:
# Insert the new dataset into the database
db.insert(dataset1, dataset2, verbose=True)

Added dataset Root (47c4d5d9-761c-432c-a446-1a1477b990e6)
Added dataset Root (996f3469-8cdb-4d68-ad4f-5e3e5ef6e3c4)


In [11]:
# Check both tables
db.connection.table("Root")

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ Root_id                              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│   100.0 │ 47c4d5d9-761c-432c-a446-1a1477b990e6 │
│    20.0 │ 996f3469-8cdb-4d68-ad4f-5e3e5ef6e3c4 │
└─────────┴──────────────────────────────────────┘

In [12]:
db.connection.table("nested")

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value     ┃ nested_id                            ┃ Root_id                              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ !string                              │ string                               │
├───────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ hi there          │ 39a55ca3-9a95-42f7-974c-e72e5f2e38e6 │ 996f3469-8cdb-4d68-ad4f-5e3e5ef6e3c4 │
│ something         │ 3bf152ac-ec14-4455-9218-637766319a9f │ 47c4d5d9-761c-432c-a446-1a1477b990e6 │
│ something else    │ b7265c13-e213-4ecb-849d-d5a31b162dd7 │ 47c4d5d9-761c-432c-a446-1a1477b990e6 │
│ this is dataset 2 │ e98828d8-624c-442e-81e4-cc643c504bbc │ 996f3469-8cdb-4d68-ad4f-5e3e5ef6e3c4 │
└───────────────────┴──────────────────────────────────────┴──────────────────────────────────────┘

In [13]:
db.connection.table("some_values")

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ some_values ┃ Root_id                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string     │ string                               │
├─────────────┼──────────────────────────────────────┤
│ Hello       │ 47c4d5d9-761c-432c-a446-1a1477b990e6 │
│ World       │ 47c4d5d9-761c-432c-a446-1a1477b990e6 │
│ Whats       │ 996f3469-8cdb-4d68-ad4f-5e3e5ef6e3c4 │
│ up          │ 996f3469-8cdb-4d68-ad4f-5e3e5ef6e3c4 │
└─────────────┴──────────────────────────────────────┘